In [2]:
# --
# Plotly
import plotly as ply
import plotly.graph_objs as go
from plotly.widgets import GraphWidget
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Layout
import plotly.tools as tls
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
out_tsplot = widgets.Output()
init_notebook_mode(connected=True)

# Plot Y limits
setLimits = False
maxY = 15000
minY = 0

toshow = []
axisshow = []

def ts_show_devices(Source):
    ts_device.options = [s for s in list(records.readings[Source]['devices'].keys())]
    ts_device.source = Source
    
def ts_show_sensors(Source):
    ts_sensor_drop.options = [s for s in list(records.readings[ts_test.value]['devices'][Source]['data'].columns)]
    ts_sensor_drop.source = Source
    ts_min_date.value = records.readings[ts_test.value]['devices'][Source]['data'].index.min()._short_repr
    ts_max_date.value = records.readings[ts_test.value]['devices'][Source]['data'].index.max()._short_repr

def ts_clear_all(b):
    with out_tsplot:
        clear_output()
        del toshow[:]
        del axisshow[:]
        
def ts_clear_sensor(b):
    with out_tsplot:
        clear_output()
        d = [ts_device.source, ts_sensor_drop.source, ts_sensor_drop.value]
        print('Removing', d[0] + ' - ' + d[1] + ' - ' + d[2])
        if d in toshow:
            toshow.remove(d)

def ts_add_sensor(b):
    with out_tsplot:
        clear_output()
        d = [ts_device.source, ts_sensor_drop.source, ts_sensor_drop.value]
        
        if d not in toshow: 
            toshow.append(d)
            axisshow.append(ts_axis_drop.value)
            
        plot_data = records.readings[toshow[0][0]]['devices'][toshow[0][1]]['data'].loc[:,(toshow[0][2],)]
        list_data_primary = []
        list_data_secondary = []
        list_data_terciary = []
        list_data_cuaternary = []
        
        if b.slice_time:
            plot_data = plot_data[plot_data.index > ts_min_date.value]
            plot_data = plot_data[plot_data.index < ts_max_date.value]
        
        if len(toshow) > 1:
            for i in range(1, len(toshow)):
                plot_data = pd.merge(plot_data, records.readings[toshow[i][0]]['devices'][toshow[i][1]]['data'].loc[:,(toshow[i][2],)], left_index=True, right_index=True)
        
        print ('-------------------------------------')
        print (' Medias:\n')
        meanTable = []
        for d in toshow:
            myMean = ' ' + d[1]  + "\t" + d[2] + "\t"
            meanTable.append(myMean)   
        res = plot_data.mean()
        for i in range(len(meanTable)): print (meanTable[i] + '%.2f' % (res[i]))
        print ('-------------------------------------')
        
        print ('-------------------------------------')
        print (' Std Deviation:\n')
        stdTable = []
        for d in toshow:
            myStd = ' ' + d[1]  + "\t" + d[2] + "\t"
            stdTable.append(myStd)   
        std = plot_data.std()
        for i in range(len(stdTable)): print (stdTable[i] + '%.2f' % (std[i]))
        print ('-------------------------------------')
    
        # Change columns naming
        changed = []
        for i in range(len(plot_data.columns)):
            changed.append(toshow[i][0] + ' - '+ toshow[i][1] + ' - '+ plot_data.columns[i])
            print(plot_data.columns[i], 'added to the plot list')
        plot_data.columns = changed
        
        subplot_rows = 0
        if len(toshow) > 0:
            for i in range(len(toshow)):
                if axisshow[i]=='1': 
                    list_data_primary.append(str(changed[i]))
                    subplot_rows = max(subplot_rows,1)
                if axisshow[i]=='2': 
                    list_data_secondary.append(str(changed[i]))
                    subplot_rows = max(subplot_rows,2)
                if axisshow[i]=='3': 
                    list_data_terciary.append(str(changed[i]))
                    subplot_rows = max(subplot_rows,3)
                if axisshow[i]=='4': 
                    list_data_cuaternary.append(str(changed[i]))
                    subplot_rows = max(subplot_rows,4)
              
        
        if ts_matplotly.value == 'Plotly':
            global fig1
            fig1 = tls.make_subplots(rows=subplot_rows, cols=1, shared_xaxes=ts_synchroniseXaxis.value)
            
            for i in range(len(list_data_primary)):
                fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_primary[i]], 'type': 'scatter', 'name': list_data_primary[i]}, 1, 1)
        
            for i in range(len(list_data_secondary)):
                fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_secondary[i]], 'type': 'scatter', 'name': list_data_secondary[i]}, 2, 1)
            
            for i in range(len(list_data_terciary)):
                fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_terciary[i]], 'type': 'scatter', 'name': list_data_terciary[i]}, 3, 1)

            for i in range(len(list_data_cuaternary)):
                fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_cuaternary[i]], 'type': 'scatter', 'name': list_data_cuaternary[i]}, 4, 1)
                
            if setLimits: 
                fig1['layout'].update(height = 1000,
                                    legend=dict(x=-.1, 
                                                y=1.2,
                                               traceorder='normal',
                                                font=dict(
                                                    family='sans-serif',
                                                    size=8,
                                                    color='#000'
                                                ),
                                                bgcolor='rgba(0,0,0,0)',
                                                bordercolor='rgba(0,0,0,0)',
                                                borderwidth=0),
                                    xaxis=dict(title='Date'))
                                  
            else:
                fig1['layout'].update(height = 1000,
                                      legend=dict(x=-.1, 
                                                y=1.2,
                                               traceorder='normal',
                                                font=dict(
                                                    family='sans-serif',
                                                    size=8,
                                                    color='#000'
                                                ),
                                                bgcolor='rgba(0,0,0,0)',
                                                bordercolor='rgba(0,0,0,0)',
                                                borderwidth=0),
                                    xaxis=dict(title='Date'))
                                   
        elif ts_matplotly.value == 'Matplotlib':
            global fig
            fig, axes = plt.subplots(subplot_rows, 1, figsize=(15,10))
            
            if subplot_rows == 1:
                for i in range(len(list_data_primary)):
                    axes.plot(plot_data.index, plot_data[list_data_primary[i]], label =  list_data_primary[i])
                    axes.legend(loc='best')
                    axes.set_ylabel(ts_text_axis_y1.value)
    
            else:
                for i in range(len(list_data_primary)):
                    axes[0].plot(plot_data.index, plot_data[list_data_primary[i]], label =  list_data_primary[i])
                    axes[0].legend(loc='best')
                    axes[0].grid(visible = True)
                    axes[0].set_ylabel(ts_text_axis_y1.value)
    
                for i in range(len(list_data_secondary)):
                    axes[1].plot(plot_data.index, plot_data[list_data_secondary[i]], label =  list_data_secondary[i])
                    axes[1].legend(loc='best')
                    axes[1].grid(visible = True)
                    axes[1].set_ylabel(ts_text_axis_y2.value)
    
                for i in range(len(list_data_terciary)):
                    axes[2].plot(plot_data.index, plot_data[list_data_terciary[i]], label =  list_data_terciary[i])
                    axes[2].legend(loc='best')
                    axes[2].grid(visible = True)
                    axes[2].set_ylabel(ts_text_axis_y3.value)
                
                for i in range(len(list_data_cuaternary)):
                    axes[3].plot(plot_data.index, plot_data[list_data_cuaternary[i]], label =  list_data_cuaternary[i])
                    axes[3].legend(loc='best')
                    axes[3].grid(visible = True)
                    axes[3].set_ylabel(ts_text_axis_y3.value)
                    
            plt.xlabel('Date')
            fig.suptitle(ts_text_title.value)
            plt.grid(visible = True)
            plt.show()
            
def ts_plot_cb(b):
    with out_tsplot:
        
        if ts_matplotly.value == 'Plotly':
            clear_output()
            ply.offline.iplot(fig1)
            
def ts_save_plot(b):
    with out_tsplot:
        
        if ts_matplotly.value == 'Matplotlib':
            
            fig.savefig(ts_savePath.value + '/plot.png', dpi=300)
        else:
            print('Not an option with plotly')

            
def ts_reset_time(b):
    ts_min_date.value = records.readings[b.src.value].index.min()._short_repr
    ts_max_date.value = records.readings[b.src.value].index.max()._short_repr

ts_layout=widgets.Layout(width='330px')

# Test dropdown
ts_test = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout=ts_layout,
                        description = 'Test')

ts_test_drop = widgets.interactive(ts_show_devices, 
                                Source=ts_test, 
                                layout=ts_layout)

# Device dropdown
ts_device = widgets.Dropdown(layout=ts_layout,
                        description = 'Device')

ts_device_drop = widgets.interactive(ts_show_sensors, 
                                Source=ts_device, 
                                layout=ts_layout)

# Sensor dropdown
ts_sensor_drop = widgets.Dropdown(layout=ts_layout,
                               description = 'Channel')

# Buttons
ts_b_add = widgets.Button(description='Prepare Plot', layout=widgets.Layout(width='120px'))
ts_b_add.on_click(ts_add_sensor)
ts_b_add.slice_time = False
ts_b_clear = widgets.Button(description='Clear', layout=widgets.Layout(width='120px'))
ts_b_clear.on_click(ts_clear_sensor)
ts_b_reset_all = widgets.Button(description='Clear all', layout=widgets.Layout(width='120px'))
ts_b_reset_all.on_click(ts_clear_all)

ts_b_plot = widgets.Button(description='Show Plot', layout=widgets.Layout(width='120px'))
ts_b_plot.on_click(ts_plot_cb)

ts_b_save = widgets.Button(description='Save Plot', layout=widgets.Layout(width='120px'))
ts_b_save.on_click(ts_save_plot)

# Axis dropdown
ts_axis_drop = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value='1',
    description='Subplot:',
    disabled=False,
    layout = ts_layout
)

# Synchronise Checkbox
ts_synchroniseXaxis = widgets.Checkbox(value=True, 
                                     description='Synchronise X axis', 
                                     disabled=False, 
                                     layout=widgets.Layout(width='300px'))

ts_matplotly = widgets.RadioButtons(
    options=['Matplotlib', 'Plotly'], value='Matplotlib',
    description='Plot Type',
    disabled=False
)

# Date fields
ts_min_date = widgets.Text(description='Start date:', 
                         layout=widgets.Layout(width='330px'))
ts_max_date = widgets.Text(description='End date:', 
                         layout=widgets.Layout(width='330px'))

# Date buttons
ts_b_apply_time = ts_b_reset = widgets.Button(description='Apply dates', layout=widgets.Layout(width='100px'))
ts_b_apply_time.on_click(ts_add_sensor)
ts_b_apply_time.slice_time = True
ts_b_reset_time = ts_b_reset = widgets.Button(description='Reset dates', layout=widgets.Layout(width='100px'))
ts_b_reset_time.on_click(ts_reset_time)

ts_device_box = widgets.HBox([ts_test_drop, ts_device_drop])
ts_add_box = widgets.VBox([widgets.HBox([ts_b_add , ts_b_clear]), widgets.HBox([ts_b_reset_all, ts_b_plot])])
ts_sensor_box = widgets.VBox([ts_sensor_drop, ts_axis_drop])
ts_plot_box = widgets.HBox([ts_sensor_box, ts_add_box])

ts_time_box = widgets.HBox([ts_min_date, ts_max_date, ts_b_reset_time, ts_b_apply_time, ts_synchroniseXaxis])
ts_text_axis_y1 = widgets.Text(description='Text axis Y1', layout=ts_layout)
ts_text_axis_y2 = widgets.Text(description='Text axis Y2', layout=ts_layout)
ts_text_axis_y3 = widgets.Text(description='Text axis Y3', layout=ts_layout)
ts_text_axis_y4 = widgets.Text(description='Text axis Y4', layout=ts_layout)
ts_text_title = widgets.Text(description='Text title', layout=ts_layout)
ts_title_box = widgets.HBox([widgets.VBox([ts_text_axis_y1, ts_text_axis_y3]), widgets.VBox([ts_text_axis_y2, ts_text_axis_y4])])
ts_savePath = widgets.Text(description='Save Path', layout = widgets.Layout(width='540px'))
ts_save_box = widgets.HBox([ts_savePath, ts_b_save])
ts_root_box = widgets.VBox([ts_matplotly, ts_time_box, ts_device_box, ts_plot_box, ts_title_box, ts_text_title, ts_save_box])

display(ts_root_box)
display(out_tsplot)

NameError: name 'widgets' is not defined